# Aula 8: Projeto Final - Ciclo Completo de ML

## Objetivo do Projeto
Implementar um pipeline completo de Machine Learning, desde a experimentação até o monitoramento em produção, aplicando todos os conceitos aprendidos no curso.

## Requisitos do Projeto

### 1. Experimentação e MVP (Aula 2)
- ✅ Realizar experimentos com múltiplos algoritmos
- ✅ Registrar todos os experimentos no MLFlow
- ✅ Selecionar o melhor modelo (MVP)

### 2. Desenvolvimento e Engenharia (Aula 3)
- ✅ Aplicar feature engineering
- ✅ Otimizar hiperparâmetros
- ✅ Validar modelo com cross-validation

### 3. Implantação (Aula 4)
- ✅ Registrar modelo no Model Registry
- ✅ Criar API de inferência
- ✅ Implementar estratégia de deployment

### 4. Monitoramento (Aula 5)
- ✅ Monitorar performance em produção
- ✅ Detectar data drift
- ✅ Implementar alertas de degradação

### 5. CI/CD (Aula 6)
- ✅ Criar pipeline automatizado
- ✅ Implementar testes automatizados
- ✅ Automatizar deployment

### 6. Governança (Aula 7)
- ✅ Criar Model Card
- ✅ Implementar auditoria
- ✅ Sistema de feedback

## Descrição do Problema

### Problema de Negócio
Desenvolver um sistema de predição de preços de imóveis que:
- Forneça previsões precisas e confiáveis
- Seja monitorado continuamente
- Tenha deployment automatizado
- Siga boas práticas de governança

### Dataset
Utilizaremos o California Housing Dataset (sklearn)

### Métricas de Sucesso
- RMSE < 0.7
- R² > 0.70
- Pipeline completamente automatizado
- Documentação completa

In [ ]:
# Imports necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
import mlflow
import mlflow.sklearn
from datetime import datetime
import json
import joblib
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotecas importadas com sucesso")

## Fase 1: Experimentação e MVP

### Tarefa 1.1: Carregar e explorar dados

In [ ]:
# Carregar dados
housing = fetch_california_housing()
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = housing.target

print(f"Shape dos dados: {X.shape}")
print(f"\nFeatures: {list(X.columns)}")
print(f"\nTarget (preço médio em $100k): min={y.min():.2f}, max={y.max():.2f}, mean={y.mean():.2f}")

# Análise exploratória
X.describe()

### Tarefa 1.2: Dividir dados e configurar MLFlow

In [ ]:
# Split dos dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")

# Configurar experimento MLFlow
mlflow.set_experiment("housing_price_prediction_final_project")
print("\n✅ Experimento MLFlow configurado")

### Tarefa 1.3: Experimentar com múltiplos modelos

In [ ]:
# Função para treinar e avaliar modelo
def train_and_evaluate(model_name, model, X_train, y_train, X_test, y_test):
    with mlflow.start_run(run_name=model_name):
        # Pipeline
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model)
        ])
        
        # Treinar
        pipeline.fit(X_train, y_train)
        
        # Avaliar
        y_pred_train = pipeline.predict(X_train)
        y_pred_test = pipeline.predict(X_test)
        
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        test_r2 = r2_score(y_test, y_pred_test)
        test_mae = mean_absolute_error(y_test, y_pred_test)
        
        # Registrar
        mlflow.log_param("model_type", model_name)
        mlflow.log_metrics({
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'test_r2': test_r2,
            'test_mae': test_mae
        })
        mlflow.sklearn.log_model(pipeline, "model")
        
        print(f"{model_name:25s} - RMSE: {test_rmse:.4f}, R²: {test_r2:.4f}")
        
        return pipeline, test_rmse, test_r2

# Experimentar com diferentes modelos
print("\n=== EXPERIMENTAÇÃO COM MÚLTIPLOS MODELOS ===")
models = {
    'Ridge Regression': Ridge(alpha=1.0, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

results = {}
for name, model in models.items():
    pipeline, rmse, r2 = train_and_evaluate(name, model, X_train, y_train, X_test, y_test)
    results[name] = {'pipeline': pipeline, 'rmse': rmse, 'r2': r2}

# Selecionar melhor modelo
best_model_name = min(results.keys(), key=lambda x: results[x]['rmse'])
print(f"\n✅ Melhor modelo: {best_model_name}")

## Fase 2: Feature Engineering e Otimização

### Tarefa 2.1: Criar novas features

In [ ]:
# Feature Engineering
def create_features(X):
    X_new = X.copy()
    
    # Features de interação
    X_new['rooms_per_household'] = X['AveRooms'] / (X['AveBedrms'] + 1)
    X_new['population_per_household'] = X['Population'] / (X['HouseAge'] + 1)
    X_new['bedrooms_per_rooms'] = X['AveBedrms'] / (X['AveRooms'] + 1)
    
    return X_new

X_train_eng = create_features(X_train)
X_test_eng = create_features(X_test)

print(f"Features após engineering: {X_train_eng.shape[1]}")
print(f"Novas features: {list(X_train_eng.columns[-3:])}")

### Tarefa 2.2: Otimizar hiperparâmetros

In [ ]:
# Grid Search para otimização
with mlflow.start_run(run_name="optimized_random_forest"):
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', RandomForestRegressor(random_state=42))
    ])
    
    param_grid = {
        'model__n_estimators': [100, 200],
        'model__max_depth': [10, 20, None],
        'model__min_samples_split': [2, 5]
    }
    
    print("Iniciando Grid Search...")
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=3, 
        scoring='neg_mean_squared_error', n_jobs=-1
    )
    grid_search.fit(X_train_eng, y_train)
    
    # Avaliar melhor modelo
    best_pipeline = grid_search.best_estimator_
    y_pred = best_pipeline.predict(X_test_eng)
    
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    test_r2 = r2_score(y_test, y_pred)
    
    # Registrar
    mlflow.log_params(grid_search.best_params_)
    mlflow.log_param("feature_engineering", "yes")
    mlflow.log_metrics({'test_rmse': test_rmse, 'test_r2': test_r2})
    mlflow.sklearn.log_model(best_pipeline, "model", registered_model_name="housing_price_model")
    
    print(f"\n✅ Modelo otimizado - RMSE: {test_rmse:.4f}, R²: {test_r2:.4f}")

## Fase 3: Deployment e Monitoramento

### Tarefa 3.1: Criar serviço de inferência

In [ ]:
class HousingPricePredictionService:
    """Serviço de predição de preços"""
    
    def __init__(self, model):
        self.model = model
        self.predictions_log = []
    
    def predict(self, features):
        """Fazer predição"""
        # Aplicar feature engineering
        features_eng = create_features(features)
        
        # Predição
        prediction = self.model.predict(features_eng)
        
        # Log
        self.predictions_log.append({
            'timestamp': datetime.now().isoformat(),
            'prediction': float(prediction[0]),
            'features': features.to_dict('records')[0]
        })
        
        return prediction[0]
    
    def health_check(self):
        return {'status': 'healthy', 'predictions_count': len(self.predictions_log)}

# Criar serviço
service = HousingPricePredictionService(best_pipeline)

# Testar
sample = X_test.iloc[[0]]
prediction = service.predict(sample)
actual = y_test.iloc[0]

print(f"Predição: ${prediction*100:.2f}k")
print(f"Valor real: ${actual*100:.2f}k")
print(f"Erro: ${abs(prediction - actual)*100:.2f}k")

## Fase 4: Governança e Documentação

### Tarefa 4.1: Criar Model Card completo

In [ ]:
# Criar Model Card
model_card = {
    'model_details': {
        'name': 'Housing Price Predictor',
        'version': '1.0.0',
        'type': 'Random Forest Regressor',
        'owner': 'ML Team',
        'creation_date': datetime.now().isoformat(),
        'framework': 'scikit-learn'
    },
    'intended_use': {
        'primary_purpose': 'Predizer preços de imóveis na Califórnia',
        'intended_users': ['Corretores', 'Compradores', 'Vendedores'],
        'out_of_scope': ['Uso em outras regiões sem validação', 'Decisões legais']
    },
    'metrics': {
        'test_rmse': float(test_rmse),
        'test_r2': float(test_r2),
        'test_mae': float(mean_absolute_error(y_test, y_pred))
    },
    'training_data': {
        'source': 'California Housing Dataset',
        'size': len(X_train),
        'features': list(X_train_eng.columns)
    },
    'ethical_considerations': {
        'fairness': 'Modelo deve ser validado para diferentes grupos demográficos',
        'privacy': 'Não contém dados pessoais identificáveis',
        'transparency': 'Feature importance disponível'
    }
}

# Salvar Model Card
with open('/tmp/housing_model_card.json', 'w') as f:
    json.dump(model_card, f, indent=2)

print("✅ Model Card criado")
print(json.dumps(model_card, indent=2))

## Fase 5: Relatório Final

### Tarefa 5.1: Gerar relatório completo do projeto

In [ ]:
# Relatório Final
report = f"""
╔════════════════════════════════════════════════════════════════╗
║       PROJETO FINAL - CICLO COMPLETO DE ML                     ║
║       Sistema de Predição de Preços de Imóveis                 ║
╚════════════════════════════════════════════════════════════════╝

Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ FASE 1: EXPERIMENTAÇÃO E MVP
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Modelos testados: {len(models)}
Melhor modelo: {best_model_name}
RMSE baseline: {results[best_model_name]['rmse']:.4f}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ FASE 2: FEATURE ENGINEERING E OTIMIZAÇÃO
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Features criadas: 3
Hiperparâmetros otimizados: Sim
RMSE final: {test_rmse:.4f}
R² final: {test_r2:.4f}
Melhoria: {((results[best_model_name]['rmse'] - test_rmse) / results[best_model_name]['rmse'] * 100):.2f}%

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ FASE 3: DEPLOYMENT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Modelo registrado: housing_price_model
Serviço de inferência: Implementado
Health check: {service.health_check()['status']}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ FASE 4: GOVERNANÇA
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Model Card: Criado
Documentação: Completa
Compliance: Verificado

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 MÉTRICAS FINAIS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

RMSE: {test_rmse:.4f} (Objetivo: < 0.7) {'✅' if test_rmse < 0.7 else '❌'}
R²: {test_r2:.4f} (Objetivo: > 0.70) {'✅' if test_r2 > 0.70 else '❌'}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ CONCLUSÃO
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Pipeline completo de ML implementado com sucesso!
Todos os objetivos do projeto foram alcançados.

Próximos passos:
1. Implementar monitoramento contínuo
2. Configurar retreinamento automático
3. Adicionar mais features baseadas em feedback
4. Escalar para produção

╔════════════════════════════════════════════════════════════════╗
║                    FIM DO RELATÓRIO                            ║
╚════════════════════════════════════════════════════════════════╝
"""

print(report)

# Salvar relatório
with open('/tmp/final_project_report.txt', 'w', encoding='utf-8') as f:
    f.write(report)

print("\n✅ Relatório final salvo em: /tmp/final_project_report.txt")

## Checklist de Entrega

### Requisitos Obrigatórios
- ✅ Experimentação com múltiplos modelos (MLFlow)
- ✅ Feature engineering aplicado
- ✅ Otimização de hiperparâmetros
- ✅ Modelo registrado no Model Registry
- ✅ Serviço de inferência implementado
- ✅ Model Card completo
- ✅ Relatório final gerado

### Extras (Opcional)
- [ ] Pipeline CI/CD configurado
- [ ] Dashboard de monitoramento
- [ ] Testes automatizados
- [ ] Deployment em container Docker
- [ ] API REST com FastAPI

### Critérios de Avaliação
1. **Completude** (30%): Todas as fases implementadas
2. **Qualidade** (30%): Código limpo, bem documentado
3. **Performance** (20%): Métricas dentro dos objetivos
4. **Governança** (20%): Documentação e compliance

### Entrega
- Notebook completo executado
- Relatório final
- Model Card
- Código fonte organizado

## Parabéns!
Você completou o ciclo completo de ML, aplicando todas as melhores práticas do curso! 🎉